# PRMT-2040 Investigate EMIS placeholder data in MI

In [1]:
import pandas as pd
import numpy as np

# Import Transfers Data

In [2]:
# Import transfer files to extract whether message creator is sender or requester
# Using data generated from branch PRMT-1742-duplicates-analysis.
# This is needed to correctly handle duplicates.
# Once the upstream pipeline has a fix for duplicate EHRs, then we can go back to using the main output.
transfer_file_location = "s3://prm-gp2gp-data-sandbox-dev/transfers-duplicates-hypothesis/"
transfer_files = [
    "9-2020-transfers.parquet",
    "10-2020-transfers.parquet",
    "11-2020-transfers.parquet",
    "12-2020-transfers.parquet",
    "1-2021-transfers.parquet",
    "2-2021-transfers.parquet"
]

transfer_input_files = [transfer_file_location + f for f in transfer_files]
transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))

# In the data from the PRMT-1742-duplicates-analysis branch, these columns have been added , but contain only empty values.
transfers_raw = transfers_raw.drop(["sending_supplier", "requesting_supplier"], axis=1)

# Given the findings in PRMT-1742 - many duplicate EHR errors are misclassified, the below reclassifies the relevant data
has_at_least_one_successful_integration_code = lambda errors: any((np.isnan(e) or e==15 for e in errors))
successful_transfers_bool = transfers_raw['request_completed_ack_codes'].apply(has_at_least_one_successful_integration_code)
transfers = transfers_raw.copy()
transfers.loc[successful_transfers_bool, "status"] = "INTEGRATED"

# Correctly interpret certain sender errors as failed.
# This is explained in PRMT-1974. Eventually this will be fixed upstream in the pipeline.
pending_sender_error_codes=[6,7,10,24,30,23,14,99]
transfers_with_pending_sender_code_bool=transfers['sender_error_code'].isin(pending_sender_error_codes)
transfers_with_pending_with_error_bool=transfers['status']=='PENDING_WITH_ERROR'
transfers_which_need_pending_to_failure_change_bool=transfers_with_pending_sender_code_bool & transfers_with_pending_with_error_bool
transfers.loc[transfers_which_need_pending_to_failure_change_bool,'status']='FAILED'

# Add integrated Late status
eight_days_in_seconds=8*24*60*60
transfers_after_sla_bool=transfers['sla_duration']>eight_days_in_seconds
transfers_with_integrated_bool=transfers['status']=='INTEGRATED'
transfers_integrated_late_bool=transfers_after_sla_bool & transfers_with_integrated_bool
transfers.loc[transfers_integrated_late_bool,'status']='INTEGRATED LATE'

# If the record integrated after 28 days, change the status back to pending.
# This is to handle each month consistently and to always reflect a transfers status 28 days after it was made.
# TBD how this is handled upstream in the pipeline
twenty_eight_days_in_seconds=28*24*60*60
transfers_after_month_bool=transfers['sla_duration']>twenty_eight_days_in_seconds
transfers_pending_at_month_bool=transfers_after_month_bool & transfers_integrated_late_bool
transfers.loc[transfers_pending_at_month_bool,'status']='PENDING'
transfers_with_early_error_bool=(~transfers.loc[:,'sender_error_code'].isna()) |(~transfers.loc[:,'intermediate_error_codes'].apply(len)>0)
transfers.loc[transfers_with_early_error_bool & transfers_pending_at_month_bool,'status']='PENDING_WITH_ERROR'

# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

asid_lookup_file = "s3://prm-gp2gp-data-sandbox-dev/asid-lookup/asidLookup-Mar-2021.csv.gz"
asid_lookup = pd.read_csv(asid_lookup_file)
lookup = asid_lookup[["ASID", "MName", "NACS","OrgName"]]

transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'requesting_supplier', 'ASID': 'requesting_supplier_asid', 'NACS': 'requesting_ods_code','OrgName':'requesting_practice_name'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'sending_supplier', 'ASID': 'sending_supplier_asid', 'NACS': 'sending_ods_code','OrgName':'sending_practice_name'}, axis=1)

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

# Import MI SR Data

In [3]:
MI_Data=pd.read_csv("s3://prm-gp2gp-data-sandbox-dev/MI_athena_outputs/MI_SR_Sept_20_Feb_21.csv")
MI_Data['conversation_id']=MI_Data['ConversationID'].str.upper()
MI_Data.shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1502251, 34)

In [4]:
MI_conversations=set(MI_Data['conversation_id'].values)
spine_conversations=set(transfers['conversation_id'].values)
overlap_conversations=MI_conversations.intersection(spine_conversations)
print(f'Number of MI conversations from Athena: {len(MI_conversations)}')
print(f'Number of transfer conversations from Spine: {len(spine_conversations)}')
print(f'Number of overlapping conversations between Athena and Spine: {len(overlap_conversations)}')
print(f'How many conversations are present in Athena MI but missing from Spine Data: {len(MI_conversations)-len(overlap_conversations)}')

print('Senders of conversations which are present in Spine Data but missing from Athena MI')
non_mi_conversations_bool=~transfers['conversation_id'].isin(MI_conversations)
transfers.loc[non_mi_conversations_bool].groupby(['sending_supplier']).agg({'conversation_id':'count'})

Number of MI conversations from Athena: 1310479
Number of transfer conversations from Spine: 1343234
Number of overlapping conversations between Athena and Spine: 1309920
How many conversations are present in Athena MI but missing from Spine Data: 559
Senders of conversations which are present in Spine Data but missing from Athena MI


,conversation_id
sending_supplier,
EMIS,1320
Microtest,1029
TPP,386
Unknown,1487
Vision,29092


## Does the Athena MI pull match an equivalent pull from Splunk?
We found a discrepency between Splunk and Athena. Splunk was 1,045,674 results, while Athena was 1,502,251.
We investigate why below

In [5]:
# Investigation of whether all the same source files were used
file_names_athena = pd.read_csv("file-names-athena.csv")
file_names_splunk = pd.read_csv("file-names.csv")
print(f'Number of Splunk source files: {file_names_splunk.shape[0]}')
print(f'Number of Athena source files: {file_names_athena.shape[0]}')
file_names_splunk_set = set(file_names_splunk["source"].values)
file_names_athena_set = set(file_names_athena["path"].values)
print(f'Intersection of Files: {len(file_names_athena_set.intersection(file_names_splunk_set))}')

Number of Splunk source files: 8888
Number of Athena source files: 55249
Intersection of Files: 8888


In [6]:
comparison_frame=file_names_athena.copy()
comparison_frame['Is in Splunk']=comparison_frame['path'].isin(file_names_splunk_set)
comparison_frame['Contains Whitespace']=comparison_frame['path'].str.contains(" ")
comparison_frame.pivot_table(index='Is in Splunk',columns='Contains Whitespace',aggfunc='count',values='_col1').fillna(0).astype(int)

Contains Whitespace,False,True
Is in Splunk,,
False,160,46201
True,8888,0


Further investigation of the same queries but just for 1st March to 1st May showed they both produced the same number of events (ie rows).
So the fix that took place in February (replacing whitespaces in filenames with underscores) has corrected this issue but has not been applied retroactively. 
The 160 files without whitespaces remain a mystery and are listed below

In [7]:
not_in_splunk_no_whitespace_bool=(~comparison_frame['Is in Splunk']) & (~comparison_frame['Contains Whitespace'])
comparison_frame.loc[not_in_splunk_no_whitespace_bool,"path"]

1545     s3://prm-gp2gp-mi-data-prod-v2/2020/12/06/2eba...
1973     s3://prm-gp2gp-mi-data-prod-v2/2021/01/03/9343...
2052     s3://prm-gp2gp-mi-data-prod-v2/2020/12/06/c90f...
2408     s3://prm-gp2gp-mi-data-prod-v2/2020/12/20/779a...
2683     s3://prm-gp2gp-mi-data-prod-v2/2020/12/27/9ff4...
                               ...                        
53424    s3://prm-gp2gp-mi-data-prod-v2/2021/01/03/8e83...
53542    s3://prm-gp2gp-mi-data-prod-v2/2020/12/13/43a7...
53956    s3://prm-gp2gp-mi-data-prod-v2/2021/01/03/f0d6...
54181    s3://prm-gp2gp-mi-data-prod-v2/2021/01/03/15bb...
55173    s3://prm-gp2gp-mi-data-prod-v2/2021/01/10/e314...
Name: path, Length: 160, dtype: object

## Does the Athena MI Data have repeating conversations?

In [8]:
# How often do conversations repeat but with different values?
MI_Data.drop_duplicates()['conversation_id'].value_counts().value_counts()

1     1198815
2      109746
3        1786
4          85
5          26
6           9
7           5
8           3
10          2
9           2
Name: conversation_id, dtype: int64

In [10]:
# For just the placeholder data, how often do conversations repeat with different placeholder data?
placeholder_columns=['PlaceholdersFileTypeUnsupported', 'PlaceholdersFileDeleted',
       'PlaceholdersFileNotFound', 'PlaceholdersFileLocked',
       'PlaceholdersUndeterminedReason']
data_of_interest=MI_Data.loc[:,placeholder_columns +['conversation_id']].drop_duplicates()
data_of_interest['conversation_id'].value_counts().value_counts()

1    1310418
2         61
Name: conversation_id, dtype: int64

# Parse out EMIS placeholders

In [11]:
placeholder_data=MI_Data.loc[:,placeholder_columns +['conversation_id']].drop_duplicates()

EMIS_sender_conversation_ids=transfers.loc[transfers['sending_supplier']=='EMIS','conversation_id'].values

EMIS_sender_MI_bool=placeholder_data['conversation_id'].isin(EMIS_sender_conversation_ids)
EMIS_sender_MI_data=placeholder_data[EMIS_sender_MI_bool]

quantity_EMIS_transfers=EMIS_sender_MI_data.shape[0]

pd.concat([pd.DataFrame(EMIS_sender_MI_data[pc_column].value_counts(dropna=False)) for pc_column in placeholder_columns],axis=1).fillna(0).div(quantity_EMIS_transfers).multiply(100).round(2).head(10)

,PlaceholdersFileTypeUnsupported,PlaceholdersFileDeleted,PlaceholdersFileNotFound,PlaceholdersFileLocked,PlaceholdersUndeterminedReason
0.0,86.97,100.0,97.32,100.0,98.55
1.0,2.10,0.0,1.36,0.0,0.73
2.0,1.32,0.0,0.49,0.0,0.25
3.0,1.01,0.0,0.24,0.0,0.12
4.0,0.78,0.0,0.15,0.0,0.08
5.0,0.68,0.0,0.09,0.0,0.05
6.0,0.58,0.0,0.06,0.0,0.04
7.0,0.50,0.0,0.04,0.0,0.03
8.0,0.45,0.0,0.03,0.0,0.02
9.0,0.39,0.0,0.02,0.0,0.02
